# IMDB Sentiment Detection using TensorFlow

Import Library/Package.

In [98]:
import tensorflow as tf
from tensorflow import keras

Load IMDB Keras Dataset, then split the loaded data into train and test.

In [99]:
imdb = keras.datasets.imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

Load provided word/dictionary mapping.

In [100]:
word_index = imdb.get_word_index()

Reserve 4 indicies, pushing the rest of words' index 3 steps ahead.

In [101]:
word_index = {k: (v + 3) for k, v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2
word_index["<UNUSED>"] = 3

Resize reviews to same length.

In [102]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data, value=word_index["<PAD>"], padding='post', maxlen=256)
test_data = keras.preprocessing.sequence.pad_sequences(test_data, value=word_index["<PAD>"], padding='post', maxlen=256)

Create model framework.

In [103]:
vocab_size = 10000
model = keras.Sequential([keras.layers.Embedding(vocab_size, 16), keras.layers.GlobalAveragePooling1D(), keras.layers.Dense(16, activation=tf.nn.relu), keras.layers.Dense(1, activation=tf.nn.sigmoid)])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

Split the data to validation and train.

In [104]:
x_val, y_val = train_data[:10000], train_labels[:10000]
partial_x_train, partial_y_train = train_data[10000:], train_labels[10000:]

Fitting.

In [105]:
model.fit(partial_x_train, partial_y_train, batch_size=512, epochs=30, validation_data=(x_val, y_val))
results = model.evaluate(test_data, test_labels)
# print(results)

Epoch 1/30


30/30 [==============================] - 2s 37ms/step - loss: 0.6922 - acc: 0.5220 - val_loss: 0.6904 - val_acc: 0.6514
Epoch 2/30
30/30 [==============================] - 1s 20ms/step - loss: 0.6870 - acc: 0.7074 - val_loss: 0.6835 - val_acc: 0.6622
Epoch 3/30
30/30 [==============================] - 0s 12ms/step - loss: 0.6753 - acc: 0.7206 - val_loss: 0.6683 - val_acc: 0.7494
Epoch 4/30
30/30 [==============================] - 0s 15ms/step - loss: 0.6544 - acc: 0.7686 - val_loss: 0.6441 - val_acc: 0.7686
Epoch 5/30
30/30 [==============================] - 0s 13ms/step - loss: 0.6223 - acc: 0.7943 - val_loss: 0.6098 - val_acc: 0.7846
Epoch 6/30
30/30 [==============================] - 0s 10ms/step - loss: 0.5804 - acc: 0.8084 - val_loss: 0.5683 - val_acc: 0.8017
Epoch 7/30
30/30 [==============================] - 0s 11ms/step - loss: 0.5328 - acc: 0.8279 - val_loss: 0.5253 - val_acc: 0.8171
Epoch 8/30
30/30 [==============================] - 0s 11ms/step - loss: 0.4844 - acc: 0.8451 

Define a function to translate the review back to human-readable.

In [106]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text if i != 0])

Evaluate.

In [107]:
COUNT = 200
test_data_print, test_labels_print = test_data[:COUNT], test_labels[:COUNT]
predictions = model.predict(test_data_print)

true_positives = 0
false_positives = 0
false_negatives = 0

for i in range(COUNT):
    print("Review: " + decode_review(test_data_print[i]))
    pred = predictions[i][0]
    value = test_labels_print[i]
    print("Prediction / Value: " + str(pred) + " / " + str(value))
    if (pred < 0.5) != (value < 0.5):
        print('\x1b[1;31mIncorrect prediction')
    else:
        print('\x1b[1;32mCorrect prediction')
    print('\x1b[0m')
    if pred >= 0.5 and value == 1:
        true_positives += 1
    elif pred >= 0.5 and value == 0:
        false_positives += 1
    elif pred < 0.5 and value == 1:
        false_negatives

precision = true_positives/(true_positives+false_positives)
recall = true_positives/(true_positives+false_negatives)
f1 = (2*precision*recall)/(recall+precision)

print(f'Precisions:\t{precision}')
print(f'Recalls:\t{recall}')
print(f'F2:\t{f1}')


7/7 [==============================] - 0s 1ms/step
Review: <START> please give this one a miss br br <UNK> <UNK> and the rest of the cast rendered terrible performances the show is flat flat flat br br i don't know how michael madison could have allowed this one on his plate he almost seemed to know this wasn't going to work out and his performance was quite <UNK> so all you madison fans give this a miss
Prediction / Value: 0.13701907 / 0
Correct prediction

Review: a lot of patience because it focuses on mood and character development the plot is very simple and many of the scenes take place on the same set in frances <UNK> the sandy dennis character apartment but the film builds to a disturbing climax br br the characters create an atmosphere <UNK> with sexual tension and psychological <UNK> it's very interesting that robert altman directed this considering the style and structure of his other films still the trademark altman audio style is evident here and there i think what really 